# Routing Analysis

In [7]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.lines import Line2D
from pathlib import Path
from statsmodels.nonparametric.smoothers_lowess import lowess


# Config
day = 170
sensitivity_factor = 1.0

in_dir = Path(f'../../export/{day}/{sensitivity_factor}/exportdata')
out_dir = Path(in_dir.parent / 'analysis/route_level')
out_dir.mkdir(parents=True, exist_ok=True)

tod_list = ['10am', '1pm', '4pm', '7pm']
colors = [
    'yellowgreen',
    'darkgoldenrod',
    'plum',
    'tab:blue',
]
default_type = 'shortest'
optimized_type = 'shaded'

palette = {
    f'{default_type} route': 'darkgrey',
    f'{optimized_type} route at {tod_list[0]}': colors[0],
    f'{optimized_type} route at {tod_list[1]}': colors[1],
    f'{optimized_type} route at {tod_list[2]}': colors[2],
    f'{optimized_type} route at {tod_list[3]}': colors[3],
}

palette_sl = {
    'long': 'black',
    'short': 'darkturquoise',
}

sns.set_theme(style='whitegrid')
sns.set_context('paper')

xlim = (-2, 102)
ylim = (-2, 102)
ylim_neg = (2, -102)

In [22]:
# Helper functions
def subfolder(out_dir, name):
    """Returns the respective folder path and creates subfolders in out_dir with timestamps if not existent already"""
    out_folder = out_dir / name
    out_folder.mkdir(exist_ok=True)
    return out_folder

def tod_format(tod):
    """Returns the formatted time of day string"""
    return tod[:-2] + ' ' + tod[-2:].upper()

## Preprocessing

In [ ]:
df = gpd.read_feather(in_dir / 'all_routes_statistics.feather')
print(df.columns)
print('Total routes: ', len(df))

In [ ]:
df_u0 = df[df['len_diff_rel'] < 0]
print(len(df_u0), len(df))
df_u0.value_counts('route_type')

In [ ]:
df_0 = df[df['len_diff_rel'] == 0.0]
print(len(df_0), len(df))
df_0.value_counts('route_type')

In [ ]:
df_o0 = df[df['len_diff_rel'] > 0]
print(len(df_o0), len(df))
df_o0.value_counts('route_type')

### Relative Length Increase vs. Solar Exposure Area Reduction 

In [10]:
x_value = 'len_diff_rel'
x_val_label='Length increase [%]'

y_value = 'sol_expo_reduction'
y_val_label='Solar exposure reduction [%]'

In [ ]:
for tod, color in zip(tod_list, colors):
    # extract row where sol_expo is not nan
    dfex = df[df[f'sol_expo_{tod}'].notna()]
    dfex = dfex[dfex['len_diff_rel'] > 0]
    dfex_heal = dfex[dfex['route_type'] == f'{optimized_type} route at {tod}']

    # calculate mean values
    xmean_heal = dfex_heal[x_value].mean()
    ymean_heal = dfex_heal[y_value].mean()

    # create Jointgrid instance and add scatter plot and kde plot
    g = sns.JointGrid(data=dfex_heal, x=x_value, y=y_value, ylim=ylim, xlim=xlim, height=5, marginal_ticks=False)
    g.ax_joint.plot((0, 100), (0, 100), color='silver', linestyle=':', linewidth=1)
    g.plot_joint(sns.scatterplot, alpha=0.66, s=5, color='silver')
    g.plot_marginals(sns.kdeplot, cut=0, color=color)
    # Compute a lowess smoothing of the data
    smoothed = lowess(exog=dfex_heal[y_value], endog=dfex_heal[x_value], frac=0.2)
    # add mean values as lines
    g.refline(x=xmean_heal, y=ymean_heal, color=color, linewidth=1)
    # add smoothed line
    g.ax_joint.plot(smoothed[:, 1], smoothed[:, 0], color=color, lw=2)
    # Invert the y-axis
    g.ax_joint.invert_yaxis()
    # Set custom ticks
    g.ax_joint.set_xticks(np.arange(0, 101, 20))
    g.ax_joint.set_yticks(np.arange(0, 101, 20))

    # set labels
    g.set_axis_labels(x_val_label, y_val_label, fontsize=12)

    # Add legend manually with line handles
    line = Line2D([0], [0], color=color, lw=2)
    g.ax_joint.legend([line], [tod_format(tod)], fontsize=10, loc='upper right')

    plt.tight_layout()
    plt.savefig(subfolder(out_dir, 'length_vs_sol_expo') / f'lowess_{tod}.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# create Jointgrid instance and add scatter plot and kde plot
g = sns.JointGrid(data=df, x=x_value, y=y_value, ylim=ylim, xlim=xlim, height=5, marginal_ticks=False)
g.ax_joint.plot((0, 100), (0, 100), color='silver', linestyle=':', linewidth=1)

for tod, color in zip(tod_list, colors):
    # extract row where sol_expo is not nan
    dfex = df[df[f'sol_expo_{tod}'].notna()]
    dfex = dfex[dfex['len_diff_rel'] > 0]
    dfex_heal = dfex[dfex['route_type'] == f'{optimized_type} route at {tod}']

    # calculate mean values
    xmean_heal = dfex_heal[x_value].mean()
    ymean_heal = dfex_heal[y_value].mean()
    print(f'{tod}: xmean={xmean_heal}, ymean={ymean_heal}')

    # Compute a lowess smoothing of the data
    smoothed = lowess(exog=dfex_heal[y_value], endog=dfex_heal[x_value], frac=0.2)

    # Add KDE plots to marginals
    sns.kdeplot(data=dfex_heal, x=x_value, ax=g.ax_marg_x, color=color, cut=0)
    sns.kdeplot(data=dfex_heal, y=y_value, ax=g.ax_marg_y, color=color, cut=0)

    # Add smoothed line
    g.ax_joint.plot(smoothed[:, 1], smoothed[:, 0], color=color, lw=2, label=tod_format(tod))

    # Add mean values as lines
    g.refline(x=xmean_heal, y=ymean_heal, color=color, linestyle='--', linewidth=1)

# Custom Axes
g.ax_joint.invert_yaxis()

# Set custom ticks
g.ax_joint.set_xticks(np.arange(0, 101, 20))
g.ax_joint.set_yticks(np.arange(0, 101, 20))

# set labels
g.set_axis_labels(x_val_label, y_val_label, fontsize=12)

# Add legend
handles, labels = g.ax_joint.get_legend_handles_labels()
g.ax_joint.legend(handles=handles, labels=labels, fontsize=10, loc='upper right')

plt.tight_layout()
plt.savefig(subfolder(out_dir, 'length_vs_sol_expo') / f'lowess_alltimes.png', dpi=300, bbox_inches='tight')
plt.show()

## Boxenplot: Solar Exposure

In [ ]:
fig, axes = plt.subplots(ncols=4, nrows=1, figsize=(12, 5), sharey=True)

for col, (tod, ax) in enumerate(zip(tod_list, axes)):
    dfex = df[df[f'sol_expo_{tod}'].notna()]
    g = sns.boxenplot(data=dfex, x='route_type', y=f'sol_expo_{tod}', palette=palette, ax=ax)
    g.set_xlabel('')
    g.set_ylabel('Solar exposure [%]', fontsize=12) if col == 0 else g.set_ylabel('')
    g.set_ylim(-2, 102)
    g.set_title(tod_format(tod), fontsize=12)
    median_default = dfex[dfex['route_type'] == f'{default_type} route'][f'sol_expo_{tod}'].median()
    median_time = dfex[dfex['route_type'] == f'{optimized_type} route at {tod}'][f'sol_expo_{tod}'].median()
    max_default = dfex[dfex['route_type'] == f'{default_type} route'][f'sol_expo_{tod}'].max()
    max_time = dfex[dfex['route_type'] == f'{optimized_type} route at {tod}'][f'sol_expo_{tod}'].max()
    min_default = dfex[dfex['route_type'] == f'{default_type} route'][f'sol_expo_{tod}'].min()
    min_time = dfex[dfex['route_type'] == f'{optimized_type} route at {tod}'][f'sol_expo_{tod}'].min()
    g.text(0, median_default, f'{round(median_default, 2)}', ha='center', va='center', fontweight='bold', fontsize=10, color='white')
    g.text(1, median_time, f'{round(median_time, 2)}', ha='center', va='center', fontweight='bold', fontsize=10, color='white')
    g.set_xticklabels([f'{default_type.title()}\n\n{max_default}\n{min_default}', f'Shaded\n\n{max_time}\n{min_time}'], fontsize=12)

plt.tight_layout()
plt.savefig(out_dir / f'sol_expo.png', dpi=300, bbox_inches='tight')
plt.show()